![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx  `mistralai/mixtral-8x7b-instruct-v01` and `meta-llama/llama-3-8b-instruct` to analyze car rental customer satisfaction from text

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.
- กรุณาใช้ Projects และข้อมูล ที่เตรียมไว้ให้ใน watsonx


## Notebook content

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

แบบฝึกหัดนี้ประกอบไปด้วยโค้ดและขั้นตอนการใช้งาน WatsonX เพื่อการทำ sentiment analysis
มีการสอนใช้ คำสั่งเพื่อการ ดึงเดต้า เทสโมเดล และการให้คะแนนโมเดล 

Some familiarity with Python is helpful. This notebook uses Python 3.11.
แบบฝึกหัดนี้ใช้ Python 3.11 เหมาะสำหรับผู้มีพื้นฐาน Python

## Learning goal

The goal of this notebook is to demonstrate how to use `ibm-mistralai/mixtral-8x7b-instruct-v01-q` and  `ibm/meta-llama/llama-3-8b-instruct` model to analyze customer satisfaction from text.

เป้าหมายของแบบฝึกหัดนี้คือ เพื่อสาธิตวิธีการใช้โมเดล `ibm-mistralai/mixtral-8x7b-instruct-v01-q` และ `ibm/meta-llama/llama-3-8b-instruct` เพื่อวิเคราะห์ความพึงพอใจของลูกค้าจากข้อความ

## Contents

This notebook contains the following parts:
แบบฝึกหัดนี้ประกอบไปด้วย:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on Watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

ก่อนที่ทุกท่านจะเริ่มทำแบบฝึกหัดนี้ กรุณาทำการติดตั้ง ตามวิธีการนี้ก่อน:


-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


-  สร้าง <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (สามารถใช้แบบไม่มีค่าใช้จ่าย และรายละเอียด การติดตั้งเป็นดังนี้ <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">ข้อมูลเพิ่มเติม</a>).


### Install and import the `datasets` and dependecies
### ดาวน์โหลด `datasets` และติดตั้งแพคเกจที่เกี่ยวข้อง

In [ ]:
# %pip install wget | tail -n 1
# %pip install datasets | tail -n 1
# %pip install scikit-learn | tail -n 1
# %pip install "ibm-watsonx-ai" | tail -n 1
# # %pip install "ibm-watson-machine-learning>=1.0.326" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

ขั้นตอนต่อไปนี้จะเป็นการระบุ credentials ที่จำเป็นต้องใช้ในการ ใช้งาน watsonx Foundation Model

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

**Action:** กรอกข้อมูล IBM cloud user API key รายละเอียดและวิธีการอยู่ข้างล่าางนี้
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [7]:
import os
import json
from dotenv import load_dotenv
from ibm_watsonx_ai.foundation_models import Model

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.


กำหนด project id เพื่อให้สามารถเชื่อมต่อแบบฝึกหนัดนี้ไปหา watsonx ได้อย่างถูกต้อง

In [9]:
#config Watsonx.ai environment
load_dotenv()
project_id = os.environ["PROJECT_ID"]
ibm_cloud_url = os.environ["IBM_CLOUD_URL"]
api_key = os.environ["WATSONX_APIKEY"]


In [10]:
#config Watsonx.ai environment

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    credentials = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

<a id="data"></a>
## Data loading

Download the `car_rental_training_data` dataset. The dataset provides insight about customers opinions on car rental. It has a label that consists of values: unsatisfied, satisfied.

ดาวน์โหลดชุดข้อมูล `car_rental_training_data` ชุดข้อมูลนี้ให้ข้อมูลความคิดเห็นของลูกค้าเกี่ยวกับการเช่ารถ มีข้อมูลป้ายกำกับที่ประกอบด้วยค่า: ไม่พอใจ, พอใจ.


In [11]:
# import wget
import pandas as pd

# filename = 'car_rental_training_data.csv'
# url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/cars-4-you/car_rental_training_data.csv'

# if not os.path.isfile(filename): 
#     wget.download(url, out=filename)

df = pd.read_csv("car_rental_training_data.csv", sep=';')
data = df[['Customer_Service', 'Satisfaction']]

Examine downloaded data.

In [12]:
data.head()

,Customer_Service,Satisfaction
0,I thought the representative handled the initi...,0
1,I have had a few recent rentals that have take...,0
2,car cost more because I didn't pay when I rese...,0
3,I didn't get the car I was told would be avail...,0
4,If there was not a desired vehicle available t...,1


Prepare train and test sets.

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
comments = list(test.Customer_Service)
satisfaction = list(test.Satisfaction)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

Model ประเภท LLM ทั้งหมด สามารถดูได้จาก ModelTypes
สามารถค้นหาข้อมูลเพิ่มเติมได้จาก [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).


In [14]:
# from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
# print([model.name for model in ModelTypes])

You need to specify `model_id` that will be used for inferencing:

ในการให้ LLM ทำงานนั้น เราเลือก `model_id` ที่เราต้องการจะใช้มัน:

In [15]:
# example model list for englist vertion
model_id = 'mistralai/mixtral-8x7b-instruct-v01'

### Defining the model parameters

**Action:** If any complications please refer to the [documentation](https://ibm.github.io/watson-machine-learning-sdk/).



### ตั้งค่า parameters ของ model

[documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames).

In [16]:
parameters = {
    'min_new_tokens': 0,
    'max_new_tokens': 1,
    'decoding_method': 'greedy',
    'repetition_penalty': 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

เลือก `Model` และตั้งค่า `parameters`

In [17]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id,
    space_id=None)

### Model's details

In [18]:
model.get_details()

{'model_id': 'mistralai/mixtral-8x7b-instruct-v01',
 'label': 'mixtral-8x7b-instruct-v01',
 'provider': 'Mistral AI',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'The Mixtral-8x7B Large Language Model (LLM) is a pretrained generative Sparse Mixture of Experts.',
 'long_description': "This model is made with AutoGPTQ, which mainly leverages the quantization technique to 'compress' the model weights from FP16 to 4-bit INT and performs 'decompression' on-the-fly before computation (in FP16). As a result, the GPU memory, and the data transferring between GPU memory and GPU compute engine, compared to the original FP16 model, is greatly reduced. The major quantization parameters used in the process are listed below.",
 'input_tier': 'class_1',
 'output_tier': 'class_1',
 'number_params': '46.7b',
 'min_shot_size': 1,
 'task_ids': ['summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'code',
  'extraction']

<a id="predict"></a>
## Analyze the satisfaction

#### Prepare prompt and generate text

#### เตรียม prompt และ เริ่มสร้างคำ

In [19]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no"""

In [20]:
prompt1 = "\n".join([instruction, "Comment:" + comments[2], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:They did not have the car I wanted.  upgraded me to a car I did not like and did not want.
Satisfied:


Analyze the sentiment for a sample of zero-shot input from the test set.

In [21]:
print(model.generate_text(prompt=prompt1))

no


### Calculate the accuracy

In [22]:
sample_size = 10
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:10]]
results = model.generate_text(prompt=prompts_batch)

In [23]:
print(prompts_batch[0] + results[0])

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:they were friendly
Satisfied:yes


In [24]:
from sklearn.metrics import accuracy_score

label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 1.0


**note watson_machine_learning : 0.7, watsonx_ai:0.8

In [25]:
print('true', y_true, '\npred', results)

true ['yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no'] 
pred ['yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no']


# Thai Language version

We have shown how sentiment classification works for the English language, now we will also showcase the same for Thai language.

In [26]:
# example mode list for thai version
# model_id = 'mistralai/mixtral-8x7b-instruct-v01'
model_id = 'meta-llama/llama-3-8b-instruct'
# model_id = 'ibm-mistralai/mixtral-8x7b-instruct-v01-q'

In [27]:
data_th = pd.read_csv('th_car_rental.csv')

In [28]:
model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id,
    space_id=None)

In [29]:
# train, test = train_test_split(data_th, test_size=0.2)
comments = list(data_th.Customer_Service_TH)
satisfaction = list(data_th.Satisfaction)

In [30]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes
Comment: พวกเขามีรถคลาสที่ฉันสั่ง
Satisfied:yes
Comment: มันสบายดี
Satisfied:yes
Comment: ต่อแถวยาวเพื่อรอรับการเช่า
Satisfied:no
"""

In [31]:
prompt1 = "\n".join([instruction, "Comment:" + comments[4], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment: การบริการลูกค้าก็ดี พวกเขาสามารถอัพเกรดชั้นเรียนถัดไปให้ฉันได้ในราคา $2 ต่อวัน...นั่นเป็นสิ่งที่ดี
Satisfied:yes
Comment: พวกเขามีรถคลาสที่ฉันสั่ง
Satisfied:yes
Comment: มันสบายดี
Satisfied:yes
Comment: ต่อแถวยาวเพื่อรอรับการเช่า
Satisfied:no

Comment:หากไม่มียานพาหนะที่ต้องการ ตัวแทนจะสำรวจตัวเลือกทั้งหมดรวมถึงคู่แข่งเพื่อช่วยในการค้นหายานพาหนะที่พร้อมใช้งาน การบริการระดับนี้ไม่ได้ทำให้ฉันกลับมาหาคู่แข่งแต่รวมถึงบริษัทด้วย เนื่องจากสิ่งนี้สะท้อนถึงคุณภาพโดยรวมของพวกเขา
Satisfied:


In [32]:
print(model.generate_text(prompt=prompt1))

yes


In [33]:
sample_size = 10
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:10]]
results = model.generate_text(prompt=prompts_batch)

In [34]:
label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 0.2


In [35]:
print('true', y_true, '\npred', results)

true ['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes'] 
pred ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to analyze car rental customer satisfaction with watsonx.ai foundation model.
 
 Check out our _[Online Documentation]()_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.